In [ ]:
import datetime
from datetime import timedelta
import os
import wget
import pandas as pd
import urllib.request

today = datetime.date.today()
yesterday = today - timedelta(days = 1)

daysinmonth = int(str(today)[-2:])
if daysinmonth < 7:
    daysinweek = daysinmonth
else:
    daysinweek = 7
    
print(daysinmonth)
print(daysinweek)

#create list that contains 7 days since today
week = []
for i in range(daysinweek):
    week.append(today - timedelta(days = i))
    week[i] = str(week[i])

print(week)

#create list that contains 30 days since today
month = []
for i in range(daysinmonth):
    month.append(today - timedelta(days = i))
    month[i] = str(month[i])

print(month)

#convert to string for managing files
yesterday = str(yesterday)
today = str(today)

#define sensor_id here
sensor_id = "12776407"
link = 'https://api-rrd.madavi.de/data_csv/csv-files/'+ today + '/data-esp8266-'+sensor_id+'-'+ today + '.csv'

#check if todays data already downloaded
def check():  
    filename = 'data-esp8266-'+sensor_id+'-'+today+'.csv'
    files = os.listdir()
    for file in files:
        if str(file) == str(filename):
            os.remove(filename) # if exist, remove it 
    return


# check if todays data is available or not, if not then download latest data of yesterday
req = urllib.request.Request(link)
try: 
    urllib.request.urlopen(req)
    check()
    filename = 'data-esp8266-'+sensor_id+'-'+today+'.csv'
    print("Todays data will be downloaded!")
    wget.download(link)
except urllib.error.URLError as e:
    print(e.reason)
    print("Todays data isn't updated on server")
    print("Downloading yesterdays data")
    today = yesterday
    filename = 'data-esp8266-'+sensor_id+'-'+today+'.csv'
    check()
    link = 'https://api-rrd.madavi.de/data_csv/csv-files/'+ today + '/data-esp8266-12776407-'+ today + '.csv'
    wget.download(link)

#read dataframe of today
df = pd.read_csv(filename,sep=';')

#convert Time string to datetime type
format = "%Y/%m/%d %H:%M:%S"
def convert_datetime(dt_string):
    dt_object = datetime.datetime.strptime(dt_string, format)
    return dt_object

df['Time'] = df['Time'].apply(convert_datetime)

#plotting latest data of one day in plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plotting(df):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df['Time'], y=df['SDS_P1'], name="PM2.5", line=dict(color='#7355A3')),
        secondary_y=False,

    )

    fig.add_trace(
        go.Scatter(x=df['Time'], y=df['SDS_P2'], name="PM10", line=dict(color='#E47988')),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="PM2.5 & PM10",
        height=600
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Time")

    # Set y-axes titles
    fig.update_yaxes(title_text="PM2.5", secondary_y=False)
    fig.update_yaxes(title_text="PM10", secondary_y=True)

    fig.show()
    
plotting(df)

#WEEKLY and MONTHLY datas fetching
#Check if folder is already exists -> if yes then delete it
import shutil
import glob

def checkfolder(folder):
    path = './' + folder
    isdir = os.path.isdir(path)
    if isdir == True:
        shutil.rmtree(path)
        print("% s removed successfully" % path)
    else:
        print("Creating " + folder)

#download weekly datas
def download_datas(folder,timeinterval):
    path = './' + folder + ''
    os.mkdir(folder)
    for date in timeinterval:
        date=str(date)
        link = 'https://api-rrd.madavi.de/data_csv/csv-files/'+ date + '/data-esp8266-12776407-'+ date + '.csv'
        wget.download(link, path)
        
def join_csvfiles(folder):
    path = './' + folder + ''
    all_files = glob.glob(os.path.join(path, "*.csv"))
    df_from_each_file = (pd.read_csv(f, sep=';') for f in all_files)
    df_merged   = pd.concat(df_from_each_file, ignore_index=True)
    df_merged.to_csv("./"+folder+"/merged.csv")

checkfolder("week")      
download_datas("week",week)    
join_csvfiles("week")

df_week = pd.read_csv("./week/merged.csv",sep=",")

#plotting latest data of one week in plotly
df_week['Time'] = df_week['Time'].apply(convert_datetime)
plotting(df_week)

#fetch and plot data of one month
checkfolder("month")
download_datas("month", month)
join_csvfiles("month")
df_month = pd.read_csv("./month/merged.csv",sep=",")
df_month['Time'] = df_month['Time'].apply(convert_datetime)
plotting(df_month)

#find last month and export to string
last_month = int(str(today)[5:7]) #10
last_month = 12 if last_month == 1 else last_month-1
if len(str(last_month)) == 1:
    last_month = "0" + str(last_month) #09

#download data of last month
last_month_link = "https://api-rrd.madavi.de/data_csv/2021/"+last_month+"/data-esp8266-"+sensor_id+"-2021-"+last_month+".zip"
checkfolder(last_month)
path = './'+last_month
os.mkdir(last_month)
wget.download(last_month_link, path)

from zipfile import ZipFile 
dir = os.listdir(path)
file = dir[0]
path = path+"/"+file
# open the zip file in read mode
with ZipFile(path, 'r') as zip: 
    # extract all files to directory
    zip.extractall(last_month)
    
os.remove(path) #delete zip file

join_csvfiles(last_month)

df_last_month = pd.read_csv("./"+last_month+"/merged.csv",sep=",",low_memory=False)
df_last_month.head()

df_last_month['Time'] = pd.to_datetime(df_last_month['Time'], errors='coerce')
plotting(df_last_month)

from bs4 import BeautifulSoup
import requests
import requests_html 

months = []
with requests_html.HTMLSession() as s:
    try:
        r = s.get('https://api-rrd.madavi.de/csvfiles.php?sensor=esp8266-12776407')
        links = r.html.links
        for link in links:
            if link[-3:] == "zip":
                months.append(link)
    except:
        pass
        
print(months)

checkfolder("all")
path = './all'
os.mkdir("all")

for link in months:
    download_link = "https://api-rrd.madavi.de/"+link
    wget.download(download_link, path)
    

days = os.listdir("./all")
for day in days:
    path="./all/"+day
    with ZipFile(path, 'r') as zip: 
        # extract all files to directory
        zip.extractall("all")
    os.remove(path)

for date in month:
    path = './all'
    date=str(date)
    link = 'https://api-rrd.madavi.de/data_csv/csv-files/'+ date + '/data-esp8266-'+sensor_id+'-'+ date + '.csv'
    wget.download(link, path)

join_csvfiles("all")
df_all = pd.read_csv("./all/merged.csv",sep=",")
df_all['Time'] = df_all['Time'].apply(convert_datetime)
plotting(df_all)

